In [9]:
'''
Musical Track Database
This application will read an iTunes export file in CSV format and produce a properly normalized database with this structure:

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);

If you run the program multiple times in testing or with different files, make sure to empty out the data before each run.

You can use this code as a starting point for your application: http://www.py4e.com/code3/tracks.zip. The ZIP file contains the tracks.csv file to be used for 
this assignment. You can export your own tracks from iTunes and create a database, but for the database that you turn in for this assignment, only use the 
tracks.csv data that is provided. You can adapt the tracks_csv.py application in the zip file to complete the assignment.

To grade this assignment, the program will run a query like this on your uploaded database and look for the data it expects to see:

SELECT Track.title, Artist.name, Album.title, Genre.name 
    FROM Track JOIN Genre JOIN Album JOIN Artist 
    ON Track.genre_id = Genre.ID and Track.album_id = Album.id 
        AND Album.artist_id = Artist.id
    ORDER BY Artist.name LIMIT 3

The expected result of the modified query on your database is: (shown here as a simple HTML table with titles)

Track                                       Artist          Album               Genre
Chase the Ace	                            AC/DC	        Who Made Who	    Rock
D.T.	                                    AC/DC	        Who Made Who	    Rock
For Those About To Rock (We Salute You)	    AC/DC	        Who Made Who	    Rock
'''
import sqlite3

trackdb = sqlite3.connect('trackdb.sqlite')
cursor = trackdb.cursor()

cursor.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Genre;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre(
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')

handle = open('tracks.csv')

for line in handle:
    line = line.strip();
    pieces = line.split(',')
    if len(pieces) < 7 : continue

    name = pieces[0]
    artist = pieces[1]
    genre = pieces[2]
    album = pieces[3]
    count = pieces[4]
    rating = pieces[5]
    length = pieces[6]

    print(name, artist, album, genre, count, rating, length)

    cursor.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) )
    cursor.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cursor.fetchone()[0]

    cursor.execute('''INSERT OR IGNORE INTO Genre (name) 
        VALUES ( ? )''', ( genre, ) )
    cursor.execute('SELECT id FROM Genre WHERE name = ? ', (genre, ))
    genre_id = cursor.fetchone()[0]

    cursor.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cursor.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cursor.fetchone()[0]

    cursor.execute('''INSERT OR REPLACE INTO Track
        (title, album_id, genre_id, len, rating, count) 
        VALUES ( ?, ?, ?, ?, ?, ? )''', 
        ( name, album_id, genre_id, length, rating, count ) )

    trackdb.commit()

Another One Bites The Dust Queen 55 Greatest Hits 100 217103 Rock
Asche Zu Asche Rammstein 79 Herzeleid 100 231810 Industrial
Beauty School Dropout Various 48 Grease 100 239960 Soundtrack
Black Dog Led Zeppelin 109 IV 100 296620 Rock
Bring The Boys Back Home Pink Floyd 33 The Wall [Disc 2] 100 87118 Rock
Circles Bryan Lee 54 Blues Is 60 355369 Funk
Comfortably Numb Pink Floyd 36 The Wall [Disc 2] 100 384130 Rock
Crazy Little Thing Called Love Queen 38 Greatest Hits 100 163631 Rock
Electric Funeral Black Sabbath 44 Paranoid 100 293015 Metal
Fat Bottomed Girls Queen 38 Greatest Hits 100 257515 Rock
For Those About To Rock (We Salute You) AC/DC 84 Who Made Who 100 353750 Rock
Four Sticks Led Zeppelin 84 IV 100 284421 Rock
Furious Angels Rob Dougan 54 The Matrix Reloaded 100 330004 Soundtrack
Gelle Bryan Lee 45 Blues Is 60 199836 Blues/R&B
Going To California Led Zeppelin 100 IV 100 215666 Rock
Gotta Move Fast Michael Loceff 19 0 100 287529 0
Grease Various 42 Grease 100 205792 Soundtrack
